### script to apply SSK BO over a locally constrained space
### we demonstrate on the task of finding proteins with low minimal free-folding energy

In [1]:
import numpy as np
import emukit
import re
from subprocess import Popen, PIPE
import subprocess
import matplotlib.pyplot as plt
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace
from emukit.core.optimization import RandomSearchAcquisitionOptimizer
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.core.loop import FixedIterationsStoppingCondition
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import our code
from boss.code.parameters.protein_base_parameter import ProteinBaseParameter
from boss.code.parameters.protein_codon_parameter import ProteinCodonParameter
from boss.code.optimizers.ProteinBaseGeneticAlgorithmAcquisitionOptimizer import ProteinBaseGeneticProgrammingOptimizer
from boss.code.optimizers.ProteinCodonGeneticAlgorithmAcquisitionOptimizer import ProteinCodonGeneticProgrammingOptimizer

from boss.code.emukit_models.emukit_bow_model import BOW_model
from boss.code.emukit_models.emukit_linear_model import linear_model
from boss.code.emukit_models.emukit_ssk_model import SSK_model
from boss.code.emukit_models.emukit_bio_features_model import BIO_Features_model

# Define problem (objective and space)

In [3]:
# define search space (a short protein to demo software)
length = 30
alphabet = ["a","c","t","g"]
space_codon = ParameterSpace([ProteinCodonParameter("string",sequence="TIKENIFGVS")])
space_base = ParameterSpace([ProteinBaseParameter("string",sequence="TIKENIFGVS")])
# protein consists of 10 amino acids (i.e 30 bases)

# define objective function (for base representations)
def objective_base(x):
    # x in 2-d numpy array of strings x =np.array([["actgg"],["gcttag"]])
    results = np.zeros((x.shape[0],1))
    for i in range(x.shape[0]):
        # call vienna RNA package
        # install binaries from https://www.tbi.univie.ac.at/RNA/
        p = subprocess.Popen('RNAfold', stdin=PIPE, stdout=PIPE) 
        string = "".join(x[i][0].split(" "))
        ans=p.communicate(string.encode())
        p.terminate()
        # collect results from output stream
        results[i][0] = float(str(ans[0]).split("(")[-1].split(")")[0])
    return results

# define objective function (for codon representations)
def objective_codon(x):
   	# x in 2-d numpy array of strings x =np.array([["'35 61 51 24 59 13 20 52 15 17'"]])    results = np.zeros((x.shape[0],1))
    results = np.zeros((x.shape[0],1))
    for i in range(x.shape[0]):
        # convert back to bases
        string = x[i][0]
        string = "".join([space_codon.parameters[0].codon_to_bases[c] for c in string.split(" ")])
        p = subprocess.Popen('RNAfold', stdin=PIPE, stdout=PIPE) 
        ans=p.communicate(string.encode())
        p.terminate()
        # collect results from output stream
        results[i][0] = float(str(ans[0]).split("(")[-1].split(")")[0])
    return results

In [4]:
# base representation is a locally constrained space of length 30 and alphabet size 4
# codon representation is a locally constrained space of length 10 and alphabet size 64

# examine sample data
print("example base representation: {}".format("".join(space_base.parameters[0].sample_uniform(1)[0][0].split(" "))))
print("example codon representation: {}".format(space_codon.parameters[0].sample_uniform(1)[0][0]))

example base representation: acaataaaggaaaacatatttggcgtcagc
example codon representation: 36 33 42 59 41 33 0 62 48 7


# Explain Methods

In [5]:
# We now demonstrate how to run all the methods described in REDACTED
#1) Linear GP applied to one-hot encoding of genes in their base representations) (i.e Taneka et al 2018)
#2) RBF GP applied to BOW representation of genes  in their base representations
#3) RBF GP applied to BOW representation of genes (+ some biologically-inspired features) in their 
#       codon representations (i.e Gonzalez et al. 2016)
#4) Our SSK approach applied to genes in their base representations
#5) Our SSK approach applied to genes in their codon representations
#6) Our split SSK approach applied to genes in their base representations
#7) Our split SSK approach applied to genes in their codon representations
#8) A purely randomized gene design loop

# Collect initial points

In [6]:
# collect initial design and initialize the search space (for both representations)
np.random.seed(42)
initial_points_count = 5

np.random.seed(42)
random_design_codon = RandomDesign(space_codon)
X_init_codon = random_design_codon.get_samples(initial_points_count)
np.random.seed(42)
random_design_base = RandomDesign(space_base)
X_init_base = random_design_base.get_samples(initial_points_count)
Y_init = objective_base(X_init_base)



FileNotFoundError: [Errno 2] No such file or directory: 'RNAfold': 'RNAfold'

# 1) Linear GP 

In [ ]:
# build BO loop
# fit BOW model
model = linear_model(space_base,X_init_base,Y_init)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinBaseGeneticProgrammingOptimizer(space_base,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_linear = BayesianOptimizationLoop(model = model, 
                                         space = space_base,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
# add loop summary
def summary(loop, loop_state):
    print("Performing BO step {}".format(loop.loop_state.iteration))
bayesopt_loop_linear.iteration_end_event.append(summary)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_linear.run_loop(objective_base, stopping_condition)

# 2) BOW of bases

In [ ]:
# build BO loop
# fit BOW model
model = BOW_model(space_base,X_init_base,Y_init,max_feature_length=5)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinBaseGeneticProgrammingOptimizer(space_base,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_BOW = BayesianOptimizationLoop(model = model, 
                                         space = space_base,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
bayesopt_loop_BOW.iteration_end_event.append(summary)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_BOW.run_loop(objective_base, stopping_condition)

# 3) BOW of codons + biollogically inspired features

In [ ]:
# build BO loop
# fit model
model = BIO_Features_model(space_base,X_init_base,Y_init,max_feature_length=5)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinBaseGeneticProgrammingOptimizer(space_base,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_BIO = BayesianOptimizationLoop(model = model, 
                                         space = space_base,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
bayesopt_loop_BIO.iteration_end_event.append(summary)

In [ ]:
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_BIO.run_loop(objective_base, stopping_condition)

# 4) SSK applied to base representations

In [ ]:
# build BO loop
# fit SSK model
model = SSK_model(space_base,X_init_base,Y_init,max_subsequence_length=5)
# choose acquisition function 
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinBaseGeneticProgrammingOptimizer(space_base,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_SSK_base= BayesianOptimizationLoop(model = model, 
                                         space = space_base,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
bayesopt_loop_SSK_base.iteration_end_event.append(summary)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_SSK_base.run_loop(objective_base, stopping_condition)

# 5) SSK applied to codon representations

In [ ]:
# build BO loop
# fit SSK model
model = SSK_model(space_codon,X_init_codon,Y_init,max_subsequence_length=5)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm  optimize acqusition function
optimizer =  ProteinCodonGeneticProgrammingOptimizer(space_codon,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)

bayesopt_loop_SSK_codon= BayesianOptimizationLoop(model = model, 
                                         space = space_codon,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)

# run BO loop for 10 steps 
bayesopt_loop_SSK_codon.run_loop(objective_codon,10)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_SSK_codon.run_loop(objective_codon, stopping_condition)

# 6) SSK applied to base representations split into 6


In [ ]:
# build BO loop
# fit SSK model
model = SSK_model(space_base,X_init_base,Y_init,num_splits=6,max_subsequence_length=5)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinBaseGeneticProgrammingOptimizer(space_base,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_SSK_base_split= BayesianOptimizationLoop(model = model, 
                                         space = space_base,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
bayesopt_loop_SSK_base_split.iteration_end_event.append(summary)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_SSK_base_split.run_loop(objective_base, stopping_condition)

# 7) SSK applied to codon representations split into 2


In [ ]:
# build BO loop
# fit SSK model
model = SSK_model(space_codon,X_init_codon,Y_init,num_splits=2,max_subsequence_length=5)
# Load core elements for Bayesian optimization
expected_improvement = ExpectedImprovement(model)
# either use genetic algorithm or random search to optimize acqusition function
optimizer =  ProteinCodonGeneticProgrammingOptimizer(space_codon,dynamic=True,population_size=100,tournament_prob=0.5,p_crossover= 0.8, p_mutation=0.1)
# optimizer = RandomSearchAcquisitionOptimizer(space,10000)
bayesopt_loop_SSK_codon_split= BayesianOptimizationLoop(model = model, 
                                         space = space_codon,
                                         acquisition = expected_improvement,
                                         acquisition_optimizer = optimizer)
bayesopt_loop_SSK_codon_split.iteration_end_event.append(summary)

In [ ]:
np.random.seed(42)
# run BO loop for 10 steps 
stopping_condition = FixedIterationsStoppingCondition(i_max = 10) 
bayesopt_loop_SSK_codon_split.run_loop(objective_codon, stopping_condition)

# 8) Perform random search

In [ ]:
np.random.seed(42)
# also see performance of random search 
#(starting from the initialization used by the other approaches)
Y_random=np.vstack([Y_init,objective_base(random_design_base.get_samples(10))])

# plot results

In [ ]:
# recall that first 5 points are a random sample shared by all the methods
# must run over multiple seeds to get an reliable idea of which algorithm is more efficient
plt.plot(np.minimum.accumulate(bayesopt_loop_linear.loop_state.Y),label="Linear")
plt.plot(np.minimum.accumulate(bayesopt_loop_BOW.loop_state.Y),label="BOW bases")
plt.plot(np.minimum.accumulate(bayesopt_loop_BIO.loop_state.Y),label="Bio Features")

plt.plot(np.minimum.accumulate(bayesopt_loop_SSK_base.loop_state.Y),label="SSK bases")
plt.plot(np.minimum.accumulate(bayesopt_loop_SSK_codon.loop_state.Y),label="SSK codons")
plt.plot(np.minimum.accumulate(bayesopt_loop_SSK_base_split.loop_state.Y),label="Split SSK bases % 6")
plt.plot(np.minimum.accumulate(bayesopt_loop_SSK_codon_split.loop_state.Y),label="Split SSK codons % 2")

plt.plot(np.minimum.accumulate(Y_random),label="Random Search")

plt.ylabel('Current best')
plt.xlabel('Iteration')
plt.legend()